<a href="https://colab.research.google.com/github/AliKhyar/Your-restaurant-at-Istanbul/blob/master/final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

/bin/bash: conda: command not found
/bin/bash: conda: command not found
Libraries imported.


In [73]:
Districts = ['Gaziosmanpaşa', 'Güngören', 'Bahçelievler', 'Bağcılar', 'Kağıthane']
Latitudes = [41.058640,41.024710,41.013180,41.034698,41.079400]
Longitudes = [28.911380,28.872700, 28.488540,28.857670,28.976370]

df = pd.DataFrame()
df['District'] = Districts
df['Latitude'] = Latitudes
df['Longitude'] = Longitudes
df.head()

,District,Latitude,Longitude
0,Gaziosmanpaşa,41.058640,28.91138
1,Güngören,41.024710,28.87270
2,Bahçelievler,41.013180,28.48854
3,Bağcılar,41.034698,28.85767
4,Kağıthane,41.079400,28.97637


In [76]:

# create map of Istanbul using latitude and longitude values
latitude, longitude = 41.008240, 28.978359
map_Istanbul = folium.Map(location=[latitude, longitude], zoom_start=10)

#add markers to map
for lat, lng, label in zip(df['Latitude'], df['Longitude'], df['District']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Istanbul)  
    
map_Istanbul
Kağıthane

In [77]:
CLIENT_ID = 'FK2ADO3XOFSGB5JEOJ4FLZA0E2Z3BP1KVRCU4QHIFVBQWC25' # your Foursquare ID
CLIENT_SECRET = '2DX1O5PPKY3JRFXCAONGYEWHRRR5KETTC32PH0R1NMK5UJJQ' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: FK2ADO3XOFSGB5JEOJ4FLZA0E2Z3BP1KVRCU4QHIFVBQWC25
CLIENT_SECRET:2DX1O5PPKY3JRFXCAONGYEWHRRR5KETTC32PH0R1NMK5UJJQ


In [114]:
# type your answer here
for label,lat, lng in zip(df['District'],df['Latitude'], df['Longitude']):
  latitude = lat
  longitude = lng
  
  LIMIT = 200

  radius = 200

  url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
      CLIENT_ID, 
      CLIENT_SECRET, 
      VERSION, 
      latitude, 
      longitude, 
      radius,
      LIMIT)
  results = requests.get(url).json()
  print("")
  print("#########################")
  print("")
  print(f"Most common at {label}:")
  print(common(results))



#########################

Most common at Gaziosmanpaşa:
Turkish Restaurant               3
Bakery                           2
Steakhouse                       2
Café                             2
Mobile Phone Shop                2
Clothing Store                   2
Pide Place                       1
Restaurant                       1
Diner                            1
Paper / Office Supplies Store    1
Hookah Bar                       1
Boutique                         1
Buffet                           1
Shoe Store                       1
Grocery Store                    1
Arcade                           1
Bagel Shop                       1
Electronics Store                1
Bar                              1
Dessert Shop                     1
Name: categories, dtype: int64

#########################

Most common at Güngören:
Turkish Restaurant                 8
Kebab Restaurant                   7
Dessert Shop                       6
Steakhouse                         6
Middle Eas

In [0]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [0]:
def common(results):
  venues = results['response']['groups'][0]['items']
      
  nearby_venues = json_normalize(venues) # flatten JSON

  # filter columns
  filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
  nearby_venues =nearby_venues.loc[:, filtered_columns]

  # filter the category for each row
  nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

  # clean columns
  nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]
  
  return nearby_venues['categories'].value_counts()
  

In [108]:
# nearby_venues.groupby('categories').count() 


Turkish Restaurant               3
Bakery                           2
Steakhouse                       2
Café                             2
Mobile Phone Shop                2
Clothing Store                   2
Pide Place                       1
Restaurant                       1
Diner                            1
Paper / Office Supplies Store    1
Hookah Bar                       1
Boutique                         1
Buffet                           1
Shoe Store                       1
Grocery Store                    1
Arcade                           1
Bagel Shop                       1
Electronics Store                1
Bar                              1
Dessert Shop                     1
Name: categories, dtype: int64

In [0]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)